In [0]:
# Classificação de Sentimentos - Tensor Flow

In [0]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split

In [3]:
!pip install category_encoders


In [4]:
!tree


/bin/bash: tree: command not found


In [0]:

df_train = pd.read_csv("https://raw.githubusercontent.com/nilberthsouza/Classificador-de-Sentimentos/master/Datasets/trainingdatasets/Train500.csv",encoding='utf-8', sep=';',engine='python')
df_train.head()
stopword = pd.read_table("https://raw.githubusercontent.com/nilberthsouza/Classificador-de-Sentimentos/master/stopwords.txt")

In [33]:
df_train.drop(['id','tweet_date','query_used'],axis=1,inplace=True)
df_train.head(100)

KeyError: ignored

In [35]:
df_train.head(80)

,tweet_text,sentiment
0,@Laranjito76 A pessoa certa para isso seria o ...,1
1,"@behin_d_curtain Para mim, é precisamente o co...",1
2,Vou fazer um video hoje... estou pensando em f...,1
3,"aaaaaaaa amei tanto essas polaroids, nem sei e...",1
4,Valoriza o coração do menininho que vc tem. El...,1
...,...,...
75,NÃO FAÇO MAIS QUESTÃO :),1
76,o primeiro a mandar a skin ganha 1 thumb :p,1
77,Amo o twitter pq posso desabafar e além de eu ...,1
78,Média? Isso choro eu em 1 mês :) https://t.co/...,1


In [0]:
stoplist = []
sentences = []
for i in range(len(stopword)):
    stoplist.append(stopword['stopwords'][i])
    
for i in range(len(df_train['tweet_text'])):
    sentence = df_train['tweet_text'][i]
    for word in stoplist:
        newword = " "+word+" "
        sentence = sentence.replace(newword," ")
        sentence = sentence.replace(newword.upper(),'')
        sentence = sentence.replace("  "," ")
    sentences.append(sentence)
    
 

agora vamos remover os emojis

In [0]:
import re
def emoji(string):
    emoji_pattern = re.compile("["
                           u"\U0001F600-\U0001F64F"  # emoticons
                           u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                           u"\U0001F680-\U0001F6FF"  # transport & map symbols
                           u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           u"\U00002702-\U000027B0"
                           u"\U000024C2-\U0001F251"
                           "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', string)


In [19]:
sentences = list(map(emoji,sentences))
sentences[0]

'@Laranjito76pessoa certa vale azevedo :)'

In [0]:
training_sentences = []
training_labels = []

training_sentences = sentences

#sentimentos negativos estão com 2 no dataset, vamos definir como 0

def twotozero(x):
  if x == 2:
    return 0
  else:
    return 1

df_train['sentiment']= df_train['sentiment'].map(twotozero)

training_labels = df_train['sentiment']

In [30]:

#training_labels[8]
df_train.head(20)

,tweet_text,sentiment
0,@Laranjito76 A pessoa certa para isso seria o ...,1
1,"@behin_d_curtain Para mim, é precisamente o co...",1
2,Vou fazer um video hoje... estou pensando em f...,1
3,"aaaaaaaa amei tanto essas polaroids, nem sei e...",1
4,Valoriza o coração do menininho que vc tem. El...,1
5,@KingJokerLeto mas amiga eu to aqui ainda :),1
6,"@RivasJairo Bravo, Jairo!!! :)))",1
7,@inquilinasls @Spotify socorro ksoaksosks acho...,1
8,"@chimekarlla gosto muito de fazenda :) , mas m...",1
9,Amanha vou ver venom :D,1


In [0]:
training_labels = np.array(training_labels)
training_labels

In [0]:
x_training, x_validation, y_training, y_validation = train_test_split(training_sentences,training_labels,test_size=0.3,random_state=17)
print(len(x_training),len(y_training), len(x_validation), len(y_validation))

In [0]:

vocab_size = 3000
embedding_dim = 10
max_length = 120
trunc_type = 'post'
oov_tok = "<UNK>"

tokenizer = Tokenizer(num_words = vocab_size, oov_token=oov_tok,)
tokenizer.fit_on_texts(x_training)
word_index = tokenizer.word_index
sequences = tokenizer.texts_to_sequences(x_training)
padded = pad_sequences(sequences, maxlen=max_length, truncating=trunc_type)

testing_sequences = tokenizer.texts_to_sequences(x_validation)
validation_padded = pad_sequences(testing_sequences,maxlen=max_length,truncating=trunc_type)

In [0]:
padded[0]

In [0]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length),
    tf.keras.layers.Dense(6, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid'),
    tf.keras.layers.Dense(1, activation='softmax'),
])

model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
model.summary()

In [0]:

num_epochs = 5
history = model.fit(padded, y_training,epochs=num_epochs,validation_data=(validation_padded,y_validation))

In [0]:
50.15 --> retirei o metodo Flatten
50.15 --> LSTM para 64
50.15 --> Primeira Layer Dense 64
50.15 --> retirado LSTM e Bidirect layer e passei pra cov1d , adcionado maxpolling1d
50.15 -->  embedding dimention de 16 pra 10
50.15 --> Dense de 64 para 6
50.15 --> GRU(32)
50.15 --> resolvido problema no trucamento